In [4]:
import requests
import json

# API endpoint for user login
LOGIN_URL = "https://piazza.e-ta.net/users/login"
# API endpoint template for getting all courses of a user
COURSES_URL_TEMPLATE = "https://piazza.e-ta.net/users/{}/courses/all"

# User credentials
email = "haosongl@usc.edu"
password = 'Lhs2016w@'

def login(email, password):
    """Login to the API and return the access token."""
    data = {"email": email, "password": password}
    response = requests.post(LOGIN_URL, json=data)
    if response.status_code == 200:
        # Assuming the API returns a token or a session ID for authenticated requests
        # This part may need to be adjusted based on the actual API response
        return response.json()
    else:
        raise Exception("Login failed!")

def get_all_course_ids(email):
    """Get all course IDs for the user."""
    # Assuming login() function returns a token or session ID in a manner that can be used here
    # Adjust the authentication method as per the actual API response structure
    auth_token = login(email, password)
    courses_url = COURSES_URL_TEMPLATE.format(email)
    # Assuming the API uses Bearer token, adjust if it uses a different method
    headers = {"Authorization": f"Bearer {auth_token}"}
    response = requests.get(courses_url, headers=headers)
    if response.status_code == 200:
        return response.json()  # A list of courses
    else:
        raise Exception("Failed to retrieve courses!")

In [5]:
def get_all_posts_json(email, course_id):
    """Fetch all posts for a specific course and return the JSON response."""
    auth_token = login(email, password)  # Assuming login() is defined elsewhere
    posts_url = f"https://piazza.e-ta.net/users/{email}/courses/{course_id}/posts/all"
    headers = {"Authorization": f"Bearer {auth_token}"}
    print(headers)
    response = requests.get(posts_url, headers=headers)
    if response.status_code == 200:
        posts = response.json()
        # Filter out any None values from the posts list before returning
        return [post for post in posts if post is not None]
    else:
        raise Exception("Failed to retrieve posts for the course.")

In [6]:
def generate_text_files_from_posts(json_response):
    with open('question.txt', 'w') as question_file, open('announcement.txt', 'w') as announcement_file:
        for post in json_response:
            if post is None:
                continue  # Skip any None entries, just in case
            if post['type'] == 'question':
                question_content = f"Question: {post['detail']['content']}\n"
                answer_content = ""
                for answer in post.get('answers', []):
                    answer_content += f"Answer: {answer['content']}\n"
                question_file.write(question_content + answer_content + "\n")
            elif post['type'] == 'note':
                announcement_content = f"Announcement: {post['detail']['content']}\n"
                announcement_file.write(announcement_content + "\n")

In [6]:
# Example of how you would use this function:
email = "haosongl@usc.edu"
password = "Lhs2016w@"
course_id = "llqyd5tpdcq34o"
try:
    json_response = get_all_posts_json(email, course_id)
    with open('posts.json', 'w') as f:
        json.dump(json_response, f, indent=4)
except Exception as e:
    print(e)

{'Authorization': "Bearer {'message': 'Login successfully'}"}
